In [1]:
import torch
from torchvision import transforms, datasets
print(torch.version.cuda) 
print(torch.backends.cudnn.version()) 

11.8
8700


In [6]:
cuda_available = torch.cuda.is_available()
cuda_available

True

In [12]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [8]:


# 定义图片转换操作
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图片大小为256x256
    transforms.ToTensor(),  # 将图片转换为Tensor
])

# 使用ImageFolder加载数据集
dataset = datasets.ImageFolder(root='E:/Kaggle/dog vs cat/train', transform=transform)

# 使用DataLoader来批量加载数据
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)





In [13]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [14]:
import torch.optim as optim

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
num_epochs = 10  # 定义训练的轮数

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个批次打印一次
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

print('Finished Training')



In [18]:
num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if i % 100 == 99:    # 每100个批次打印一次
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

print('Finished Training')


Epoch [1/2], Step [100], Loss: 0.2510
Epoch [1/2], Step [200], Loss: 0.2646
Epoch [1/2], Step [300], Loss: 0.2409
Epoch [1/2], Step [400], Loss: 0.2741
Epoch [1/2], Step [500], Loss: 0.2582
Epoch [1/2], Step [600], Loss: 0.2480
Epoch [1/2], Step [700], Loss: 0.2579
Epoch [1/2], Loss: 0.0257, Accuracy: 89.45%
Epoch [2/2], Step [100], Loss: 0.1686
Epoch [2/2], Step [200], Loss: 0.1987
Epoch [2/2], Step [300], Loss: 0.1983
Epoch [2/2], Step [400], Loss: 0.2015
Epoch [2/2], Step [500], Loss: 0.1950
Epoch [2/2], Step [600], Loss: 0.2071
Epoch [2/2], Step [700], Loss: 0.1917
Epoch [2/2], Loss: 0.0251, Accuracy: 91.99%
Finished Training


In [20]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [24]:
model = SimpleCNN()
model.load_state_dict(torch.load('./cnn.pth'))

# 将模型设置为评估模式
model.eval()

# 确保模型在正确的设备上
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [25]:
import os
from PIL import Image

test_path = 'E:/Kaggle/dog vs cat/test1/test1'
image_lists = os.listdir(test_path)
pre = []
for image_list in image_lists:
    index = os.path.splitext(image_list)[0]
    image_path = os.path.join(test_path, image_list)
    image = Image.open(image_path)
    image = image.convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs.data, 1)
    pre.append([index, predicted.item()])


TypeError: conv2d() received an invalid combination of arguments - got (Image, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
